# Motivation: Warum Dataclasses?
## Der traditionelle Weg: Viel Code für einfache Strukturen
Schauen wir uns an, wie viel Code wir für einen einfachen mathematischen Vektor brauchen:

In [1]:
class Vector2D:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
    
    # Vektoren vergleichen
    def __eq__(self, other):
        if not isinstance(other, Vector2D):
            return NotImplemented
        return self.x == other.x and self.y == other.y
    
    # Lesbare Darstellung
    def __repr__(self):
        return f"Vector2D(x={self.x}, y={self.y})"
    
    # String-Darstellung
    def __str__(self):
        return f"({self.x}, {self.y})"
        
    # Addition von Vektoren
    def __add__(self, other):
        if not isinstance(other, Vector2D):
            return NotImplemented
        return Vector2D(self.x + other.x, self.y + other.y)
    
    def length(self):
        return (self.x**2 + self.y**2)**0.5

# Verwendung
v1 = Vector2D(3, 4)
v2 = Vector2D(1, 2)
v3 = v1 + v2
print(v1 == v2)  # False
print(v1)        # (3, 4)
print(v3)        # (4, 6)

False
(3, 4)
(4, 6)


## Die elegante Lösung mit Dataclass:

In [6]:
from dataclasses import dataclass

@dataclass
class Vector2D:
    x: float
    y: float
    
    def length(self):
        return (self.x**2 + self.y**2)**0.5
    
    def __add__(self, other):
        return Vector2D(self.x + other.x, self.y + other.y)

# Verwendung genau wie oben
v1 = Vector2D(3, 4)
v2 = Vector2D(1, 2)
v3 = v1 + v2
print(v1 == v2)  # False - automatisch generiert!
print(v1)        # Vector2D(x=3, y=4) - automatisch generiert!
print(v3)        # Vector2D(x=4, y=6)

False
Vector2D(x=3, y=4)
Vector2D(x=4, y=6)


## Sehen Sie den Unterschied? Der Dataclass-Code ist:

- Etwa 70% kürzer
- Leichter zu lesen
- Weniger fehleranfällig
Hat trotzdem alle wichtigen Funktionen

# Unveränderlichkeit

Beispiel: Basisvektor e1.

In [20]:
@dataclass(frozen=True)
class Vector:
    x: float
    y: float
    
    def length(self):
        return (self.x**2 + self.y**2)**0.5

# Liste von Vektoren
vectors = [Vector(1, 1), Vector(2, 2), Vector(3, 3)]

e1 = Vector(0,1)
print(e1)
print(f"{e1.x=}")
#e1.x = 1
print(f"{e1=}")

# Diese Berechnung ist sicher, da die Vektoren nicht verändert werden können
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    lengths = list(executor.map(lambda v: v.length(), vectors))

Vector(x=0, y=1)
e1.x=0
e1=Vector(x=0, y=1)


# Dekoratoren

### Funktionen, die Funktionen zurückgeben

In [21]:
def outer_function(x):
    def inner_function(y):
        return x + y
    return inner_function

# Erstellt eine Funktion, die 5 addiert
add_five = outer_function(5)
print(add_five(3))  # Ausgabe: 8

8


In [5]:
def simple_decorator(func):
    def wrapper():
        print("Vor der Funktion")
        func()
        print("Nach der Funktion")
    return wrapper

# Verwendung mit @ Syntax
@simple_decorator
def say_hello():
    print("Hallo!")

# Äquivalent zu:
# say_hello = simple_decorator(say_hello)

say_hello()
# Ausgabe:
# Vor der Funktion
# Hallo!
# Nach der Funktion

Vor der Funktion
Hallo!
Nach der Funktion


In [3]:
def repeat(times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(times=3)
def greet(name):
    print(f"Hallo {name}!")

greet("Max")  # Wird 3x ausgeführt

Hallo Max!
Hallo Max!
Hallo Max!


In [6]:
import time

def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Funktion {func.__name__} brauchte {end_time - start_time:.4f} Sekunden")
        return result
    return wrapper

@measure_time
def calculate_sum(n):
    return sum(range(n))

# Test
print(calculate_sum(1000000))

Funktion calculate_sum brauchte 0.0208 Sekunden
499999500000


## Mehrere Dekoratoren

In [7]:
def bold(func):
    def wrapper():
        return f"<b>{func()}</b>"
    return wrapper

def italic(func):
    def wrapper():
        return f"<i>{func()}</i>"
    return wrapper

@bold
@italic
def get_text():
    return "Hallo Welt!"

print(get_text())  # Ausgabe: <b><i>Hallo Welt!</i></b>

<b><i>Hallo Welt!</i></b>


### Praktische Übung
Aufgabe: Erstellen Sie einen Decorator, der die Ausführungszeit einer Funktion misst.

In [22]:
import time

def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        print(f"Funktion {func.__name__} brauchte {end_time - start_time:.4f} Sekunden")
        return result
    return wrapper

@measure_time
def calculate_sum(n):
    return sum(range(n))

# Test
print(calculate_sum(1000000))

Funktion calculate_sum brauchte 0.0184 Sekunden
499999500000


### functools.wraps
Motivation: Doc String und Name der Funktion.

In [4]:
from functools import wraps

def my_decorator(func):
    @wraps(func)  # Wichtig: Behält Metadaten der originalen Funktion
    def wrapper(*args, **kwargs):
        """Wrapper Funktion"""
        print(f"Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(name):
    """Greets a person"""
    print(f"Hello {name}!")

# Ohne @wraps würden wir hier 'wrapper' und dessen Docstring sehen
print(greet.__name__)  # Ausgabe: 'greet'
print(greet.__doc__)   # Ausgabe: 'Greets a person'

# Vergleich ohne @wraps:
def decorator_without_wraps(func):
    def wrapper(*args, **kwargs):
        """Wrapper Funktion"""
        return func(*args, **kwargs)
    return wrapper

@decorator_without_wraps
def hello(name):
    """Says hello"""
    print(f"Hello {name}!")

print(hello.__name__)  # Ausgabe: 'wrapper'
print(hello.__doc__)   # Ausgabe: 'Wrapper Funktion'

greet
Greets a person
wrapper
Wrapper Funktion


# Dekoratoren für Klassen

In [5]:
class DatabaseOld:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        # Wird jedes Mal aufgerufen, auch wenn existierende Instanz zurückgegeben wird
        print("Initialisiere Datenbank...")

# Test
db1 = DatabaseOld()  # Druckt "Initialisiere Datenbank..."
db2 = DatabaseOld()  # Druckt "Initialisiere Datenbank..." (unerwünscht!)
print(db1 is db2)    # True, aber __init__ wurde zweimal aufgerufen

Initialisiere Datenbank...
Initialisiere Datenbank...
True


## Probleme der traditionellen Implementierung

- __init__ wird mehrfach aufgerufen -- auch wenn gar kein neues Objekt erzeugt wird
- Code für Singleton-Logik vermischt sich mit Klassen-Code
- Schwer wiederverwendbar für andere Klassen

## Lösung mit Decorator

In [23]:
def singleton(cls):
    # Dictionary für alle Singleton-Instanzen
    instances = {}
    
    # Wrapper-Funktion, die aufgerufen wird, wenn eine Instanz erstellt werden soll
    def get_instance(*args, **kwargs):
        if cls not in instances:
            # Erstelle neue Instanz, wenn noch keine existiert
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    # Gib Wrapper zurück
    return get_instance

# Test
@singleton
class Database:
    def __init__(self):
        print("Initialisiere Datenbank...")

db1 = Database()  # Druckt "Initialisiere Datenbank..."
db2 = Database()  # Druckt nichts, nutzt existierende Instanz
print(db1 is db2)  # True

Initialisiere Datenbank...
True


# Vorteile des Decorator-Ansatzes

- Trennung von Singleton-Logik und Klassencode
- Wiederverwendbar für beliebige Klassen
- Einfach zu verstehen und zu warten
- Erweiterbar um zusätzliche Funktionalität

# 